In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter
from torch.utils.data import DataLoader


In [2]:
# All of our fields have names, but we need them in numbers, so define a dictionary to convert.
global magDict
magDict = {
    'TOTUSJH': 0,
    'TOTBSQ': 1,
    'TOTPOT': 2,
    'TOTUSJZ': 3,
    'ABSNJZH': 4,
    'SAVNCPP': 5,
    'USFLUX': 6,
    'TOTFZ': 7,
    'MEANPOT': 8,
    'EPSZ': 9,
    'SHRGT45': 10,
    'MEANSHR': 11,
    'MEANGAM': 12,
    'MEANGBT': 13,
    'MEANGBZ': 14,
    'MEANGBH': 15,
    'MEANJZH': 16,
    'TOTFY': 17,
    'MEANJZD': 18,
    'MEANALP': 19,
    'TOTFX': 20,
    'EPSY': 21,
    'EPSX': 22,
    'R_VALUE': 23,
    'RBZ_VALUE': 24,
    'RBT_VALUE': 25,
    'RBP_VALUE': 26,
    'FDIM': 27,
    'BZ_FDIM': 28,
    'BT_FDIM': 29,
    'BP_FDIM': 30,
    'PIL_LEN': 31,
    'XR_MAX': 32
}

In [27]:
# Get the data from the JSON file, then return it as a tensor of input data and a list of labels
def getDataFromJSON(path="data/train_partition1_data.json", device='cpu', earlyStop=-1):
    # I might need to refactor these arguments to allow to get the test data. Problem for another day.
    # path is the path to the files, device is where to store it (CUDA), earlyStop is how many lines to 
    # read if you don't want the entire file read.
    
    # Get the dictionary to assign names to numbers
    global magDict
    
    # This dataset is heavily skewed, so we need to get the number of each type of flare.
    # This also lets us get the number of lines in the file with a sum.
    # This function also ignores any lines with a value of NaN.
    weights = counter(path, earlyStop)
    lines = np.sum(weights)
    # Check when we want to stop - the end of the file or earlier.
    if earlyStop < 0: length = lines
    else: length = min(earlyStop, lines)
    
    # Get the file and open it. 
    file = open(path)    
    
    # Declare a tensor to hold the data, and a list to hold the labels.
    # Dimensions: 0: number of entries we want. 1: the 33 fields in the data. 2: the 60 observations in each field. 
    tnsr = torch.Tensor().new_empty((length, 33, 60), device=device)
    labels = []
    flares = {'X':0, 'M':1, 'C':2, 'B':3, 'Q':4}
        
    row = -1
    for line in file:
        if 'nan' in line or "NaN" in line:
            continue
        # Load the line as a dictionary. Row is an integer place and v is a smaller dictionary.
        d: dict = json.loads(line)
        row += 1
        for _, v in d.items(): # we use the _ because we don't want the ID.
            if earlyStop > 0 and row >= earlyStop:
                # If we don't want the entire dataset, stop loading more than we want
                return tnsr, labels, weights
            if row % 100 == 0:
                print(f'Now loading event {row}/{length}')
            # append the label to our list
            labels.append(flares[v['label']])
            
            # Break each individual dictionary into dictionaries of observations
            # Key is the string in magDict, and timeDict is a dictionary of observations over time
            for key, timeDict in v['values'].items():
                # Turn our name string into a numeric value
                location = magDict[key]
                # Get the measurements out of the time series dictionary
                for timeStamp, measurement in timeDict.items():
                    tnsr[row][location][int(timeStamp)] = measurement
    print(f'{row} lines loaded.')
    # Close the file. I'm not a heathen                    
    file.close()
    # This might be a good place to perform some post processing, but that's a question for another day.
    # Famous last words.
    return tnsr, labels, weights



In [28]:
# This file has 77270 data points. 
%time train1, labels1, weights1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=10000)

Now loading event 0/10000
Now loading event 100/10000
Now loading event 200/10000
Now loading event 300/10000
Now loading event 400/10000
Now loading event 500/10000
Now loading event 600/10000
Now loading event 700/10000
Now loading event 800/10000
Now loading event 900/10000
Now loading event 1000/10000
Now loading event 1100/10000
Now loading event 1200/10000
Now loading event 1300/10000
Now loading event 1400/10000
Now loading event 1500/10000
Now loading event 1600/10000
Now loading event 1700/10000
Now loading event 1800/10000
Now loading event 1900/10000
Now loading event 2000/10000
Now loading event 2100/10000
Now loading event 2200/10000
Now loading event 2300/10000
Now loading event 2400/10000
Now loading event 2500/10000
Now loading event 2600/10000
Now loading event 2700/10000
Now loading event 2800/10000
Now loading event 2900/10000
Now loading event 3000/10000
Now loading event 3100/10000
Now loading event 3200/10000
Now loading event 3300/10000
Now loading event 3400/100

In [29]:
# This file has 93767 data points. 
%time train2, labels2, weights2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=10000)

Now loading event 0/10000
Now loading event 100/10000
Now loading event 200/10000
Now loading event 300/10000
Now loading event 400/10000
Now loading event 500/10000
Now loading event 600/10000
Now loading event 700/10000
Now loading event 800/10000
Now loading event 900/10000
Now loading event 1000/10000
Now loading event 1100/10000
Now loading event 1200/10000
Now loading event 1300/10000
Now loading event 1400/10000
Now loading event 1500/10000
Now loading event 1600/10000
Now loading event 1700/10000
Now loading event 1800/10000
Now loading event 1900/10000
Now loading event 2000/10000
Now loading event 2100/10000
Now loading event 2200/10000
Now loading event 2300/10000
Now loading event 2400/10000
Now loading event 2500/10000
Now loading event 2600/10000
Now loading event 2700/10000
Now loading event 2800/10000
Now loading event 2900/10000
Now loading event 3000/10000
Now loading event 3100/10000
Now loading event 3200/10000
Now loading event 3300/10000
Now loading event 3400/100

In [30]:
# This file has 42986 data points. 
%time train3, labels3, weights3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=10000)

Now loading event 0/10000
Now loading event 100/10000
Now loading event 200/10000
Now loading event 300/10000
Now loading event 400/10000
Now loading event 500/10000
Now loading event 600/10000
Now loading event 700/10000
Now loading event 800/10000
Now loading event 900/10000
Now loading event 1000/10000
Now loading event 1100/10000
Now loading event 1200/10000
Now loading event 1300/10000
Now loading event 1400/10000
Now loading event 1500/10000
Now loading event 1600/10000
Now loading event 1700/10000
Now loading event 1800/10000
Now loading event 1900/10000
Now loading event 2000/10000
Now loading event 2100/10000
Now loading event 2200/10000
Now loading event 2300/10000
Now loading event 2400/10000
Now loading event 2500/10000
Now loading event 2600/10000
Now loading event 2700/10000
Now loading event 2800/10000
Now loading event 2900/10000
Now loading event 3000/10000
Now loading event 3100/10000
Now loading event 3200/10000
Now loading event 3300/10000
Now loading event 3400/100

In [36]:
train2[0,0]

tensor([19.0601, 18.4611, 25.6917, 17.3464, 18.1081, 18.0577, 18.7077, 17.4748,
        17.8627, 16.9230, 22.6015, 22.5883, 18.8607, 19.9172, 23.7632, 21.3895,
        27.7625, 26.1885, 26.1678, 27.3103, 27.4704, 26.1131, 29.2025, 31.6294,
        28.8686, 27.6369, 26.0752, 20.9289, 19.3941, 17.8339, 23.0833, 19.8953,
        20.2367, 20.7759, 21.6280, 22.9786, 20.4450, 20.1092, 19.7991, 21.1444,
        16.3790, 20.6794, 16.9210, 18.0858, 16.4018, 16.2757, 17.2050, 19.5720,
        19.2548, 19.9467, 18.0555, 18.8806, 18.5460, 20.5339, 18.9813, 16.3843,
        18.5569, 17.3046, 17.9197, 14.8065])

In [38]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.smax(self.layerout(y))
        return y





In [47]:
def train(model, inputs, labels, weight, valSets, valLabels, valweight):
    # TODO: Is this right? How do I determine the weights here?
    weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=100/weight)
#     valLabels = torch.tensor(valLabels, dtype=torch.int)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    # 1/(weight+1) - makes sure we don't have any pesky zeroes
    # np.sum(weight)/weight if your learning rate is too low.
    
    # Hyperparameters
    batch = 256
    epochs = 10
    lr = 0.01
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(len(val)/4), num_workers=4)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        batch_loss = []
        for (xtrain,ytrain) in loader:
            output = model(xtrain)
            loss = lfc(output,ytrain)
            opt.zero_grad()
            loss.backward()
            opt.step()
            batch_loss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
        
        
        model.eval()
        balanced = [[],[],[],[],[]]
        batchLoss = []
        unbalanced = []
        
        for (xval,yval) in valLoader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            corrects = yval.clone().detach() == torch.argmax(output)
            unbalanced.append(np.mean([1 if correct else 0 for correct in corrects.detach()]))
            
            # TODO: figure this one out.
            for i, ans in enumerate(yval):
                balanced[ans].append(corrects[i])
        
        balanced = [np.mean(i) for i in balanced]
        balancedAccuracy = np.mean(balanced)
        
        print(f'The total balanced accuracy for validation was {balancedAccuracy}')
        print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
        print(f'The accuracy for each is {balanced}')           
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')

            
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
    return model


In [54]:
model = logRegWithHidden(60, 30)
    # print(weights1+weights2, 1.0/np.array(weights1))

[172, 1157, 6531, 6010, 63400, 72, 1393, 9130, 5159, 78013] [5.81395349e-03 8.64304235e-04 1.53115909e-04 1.66389351e-04
 1.57728707e-05]


In [48]:
print(model)
model = train(
    model,
    torch.cat((train1, train2), dim=0),
    labels1 + labels2,
    [weights1[i] + weights2[i] for i in range(5)],
    train3,
    labels3,
    weights3
)

logRegWithHidden(
  (layer1): Linear(in_features=1980, out_features=60, bias=True)
  (layer2): Linear(in_features=60, out_features=30, bias=True)
  (layerout): Linear(in_features=30, out_features=5, bias=True)
  (relu): ReLU()
  (smax): Softmax(dim=1)
  (drop): Dropout(p=0.5, inplace=False)
)
The training loss for epoch 1/10 was 1.6985065016565444
The total balanced accuracy for validation was 0.2041765621805256
The unbalanced validation accuracy is 0.4221
The accuracy for each is [0.0, 0.0, 0.0, 0.5161290322580645, 0.5047537786445636]
The validation loss was :   1/10 was 1.782215416431427
The validation loss was :   1/10 was 1.782215416431427
The training loss for epoch 2/10 was 1.7053894393051727
The total balanced accuracy for validation was 0.2041765621805256
The unbalanced validation accuracy is 0.4221
The accuracy for each is [0.0, 0.0, 0.0, 0.5161290322580645, 0.5047537786445636]
The validation loss was :   2/10 was 1.782215416431427
The validation loss was :   2/10 was 1.782215

In [42]:
print(model.parameters())
feature_extraction = [child for child in model.children()]
for line in feature_extraction:
    print(line)
    print(f'weights: {line.weight}')

<generator object Module.parameters at 0x7f8198477190>
Linear(in_features=1980, out_features=60, bias=True)
weights: Parameter containing:
tensor([[ 9.3467e-03, -7.5361e-05,  1.1365e-02,  ...,  2.1932e-02,
          2.0284e-03,  1.7351e-02],
        [ 7.7337e-04, -2.2144e-02,  1.3253e-03,  ...,  1.0807e-02,
          6.0054e-03,  2.1932e-02],
        [-1.2613e-02,  2.2360e-02,  1.8253e-02,  ...,  1.5422e-02,
          1.2585e-02,  1.7047e-03],
        ...,
        [ 1.2793e-02,  1.5726e-02, -8.2898e-03,  ..., -3.8409e-03,
          2.1281e-02,  1.3310e-02],
        [-4.8142e-06,  1.5197e-02, -9.3472e-03,  ...,  4.5212e-03,
          3.5038e-03, -1.7310e-02],
        [-5.6978e-03,  2.1071e-02,  8.0062e-03,  ..., -1.8534e-02,
         -1.3441e-03, -1.0579e-02]], requires_grad=True)
Linear(in_features=60, out_features=30, bias=True)
weights: Parameter containing:
tensor([[ 0.0801, -0.0175,  0.0943,  ..., -0.0980,  0.0416,  0.1004],
        [-0.0074, -0.0828,  0.0167,  ...,  0.0483, -0.081

ModuleAttributeError: 'ReLU' object has no attribute 'weight'